In [1]:
import pandas as pd
import pickle
import os
import numpy as np
import random
from colog.colog import colog
c = colog()

In [2]:
def load_data_one_stock(path):
  load_data = pd.DataFrame()
  try:
    file = open(path, 'rb')
    load_data= pickle.load(file)
    file.close()
  except Exception as e:
    print(e)
  return load_data

In [3]:
# SETTINGS
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 1000
commission_value = 0.001
trainig_part = 0.5
test_part = 0.5

Exlude companies with bad ration and retraining!!!

Currten best settings:
{'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10}
-3458.0764813327787 495


In [4]:
## Load data

# file_name = 'df_stock_AAPL_period2y_interval1h.pkl'
# path = os.path.join(parent_path, folder_saved_models, file_name)
## stock_name = item[str.find(item, 'df_') + 3 : str.find(item, '_20')]
# df = load_data_one_stock(path)
## df.plot()
## print(df)

In [5]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
 
  if gmax == reference_point:
    result = False
  elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
    result = True
  else:
    result = False

  return result

def number_red_candles(df, i, k=11):

  if i < k:
    number_red_candles = (df['ha_colour'][0 : i] == 'red').sum()
  else:
    number_red_candles = (df['ha_colour'][i - k : i] == 'red').sum()
  return number_red_candles

In [107]:
def one_stock_simulation(df, ticker, mode, _gain_coef = 1.005, _lose_coef = 0.95, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test

    Returns:
      profits, wins, loses 
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'number wins': 0,
                'proportion': 0,
                'profit': 0,
                'transactions': [] }  
  last_top = df['close'].iloc[0]
  last_top_time = df['close'].index[0]
  last_top_i = 0
  reverse_flag = False
  sell_finished = True
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'full': 
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'test':
    range_ = range(int(df.shape[0] * trainig_part) + 1, df.shape[0] - 1)

  for i in range_:

    # dynaminc profit/loose coefficient
    if i > 200:
      if df['close'].iloc[i] / df['close'].iloc[i - 200] > bull_trend_coef: # bull trend
        gain_coef = 1.02
        lose_coef = 0.95  # not equal to 1 - k (DO Different Expertiments)
      else: # bear trend
        gain_coef = _gain_coef
        lose_coef = _lose_coef  # used to be 0.95 16/08/2024
        # print(f'lose coef {lose_coef}')
    else: # undefined trend
      gain_coef = 1.02
      lose_coef = 0.98

    if sell_finished == False and df['high'].iloc[i] / buy_price > gain_coef:
      sell_price = float(df['high'].iloc[i])
      history_log['number success'] += 1
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number wins'] += 1
      history_log['transactions'].append({
        'type': 'success',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2), 
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['high'].index[i],
        'buy_lose_coef': buy_lose_coef,
        'buy_gain_coef': buy_gain_coef,
     })
      sell_finished = True

    if sell_finished == False and df['low'].iloc[i] / buy_price < lose_coef :
      sell_price = float(df['low'].iloc[i])
      sell_finished = True
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number fails'] += 1
      history_log['transactions'].append({
        'type': 'fail',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2),
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['low'].index[i],
        'buy_lose_coef': buy_lose_coef,
        'buy_gain_coef': buy_gain_coef,
      })

    # KEY parameters: RIV, prt, last_top_ratio
    # buy_time_cond = (df.index[i]).hour <= 15 and (df.index[i]).hour >= 11

    buy_ratio = float(last_top -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])
    if df['ha_pct'].iloc[i] > RIV \
      and reverse_flag==False \
      and sell_finished == True \
      and last_top / df['open'].iloc[i] > last_top_ratio \
      and i - last_top_i > distance_from_last_top \
      and buy_time_cond \
      and buy_ratio > buy_ratio_border \
      and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)) \
      and number_red_candles(df,i) > 6:

      reverse_flag = True
      # buy_price = float(df['close'].iloc[i])
      buy_price = float(df['open'].iloc[i + 1])
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      buy_gain_coef = gain_coef
      buy_lose_coef = lose_coef
      sell_finished = False

    # last top and reverse flag
    if df['ha_colour'].iloc[i] == 'red' \
      and df['ha_colour'].iloc[i - 1] == 'green'\
      and df['ha_colour'].iloc[i - 2] == 'green'\
      and df['ha_pct'].iloc[i - 1] > 0.1 \
      and df['ha_pct'].iloc[i - 2] > 0.1:
  
      last_top = df['high'].iloc[i - 1]
      last_top_i = i - 1
      reverse_flag = False
  
  history_log['proportion'] =   history_log['number wins'] / ( history_log['number fails'] + 1)

  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(history_log)

    df_tran = pd.DataFrame(history_log['transactions'])
    print('Statictic for buy gain coef = 1.02')
    print(df_tran.loc[df_tran['buy_gain_coef'] == 1.02]['type'].value_counts())
    print(f'Statictic for buy_gain coef = {_gain_coef}')
    print(df_tran.loc[df_tran['buy_gain_coef'] == _gain_coef]['type'].value_counts())
  
  if print_trans_history:
    for tran in history_log['transactions']:
      if tran['buy_gain_coef'] == 1.02:
        c.print(tran, color='green')
      else:
        c.print(tran, color='blue')

  return history_log['profit'], history_log['number wins'], history_log['number fails']


In [86]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12

# best_params = {'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 5, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
# after last check
best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}

for param in best_params:
  vars()[param] = best_params[param]

parameters_list_search = {
  'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100],
  'distance_from_last_top': [0,2,3,5],
  'last_top_ratio': [1, 1.001, 1.002, 1.003],
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
  'buy_ratio_border' : [0,3,5,7,9,10],
  'bull_trend_coef': [1.07, 1.12]
}

# parameters_list_search = {
#   'bull_trend_coef': np.arange(1.01, 1.25, 0.01)
# }

In [84]:
# BEST PARAMETERS OPTIMIZATION  and OPTIMAL STOCK LIST
best_profit = -10**5
best_profit_improved = True
optimize_profit = True
ticker_stats = {}
stock_name_list_opt = stock_name_list
# for parameter in parameters_list_search:
while best_profit_improved:
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    for value in parameters_list_search[parameter]:
      vars()[parameter] = value
      c.print(f'parametr is {parameter} value is {value}:', color='white')
      total_profit = 0
      total_loses = 0
      total_wins = 0
      for ticker in stock_name_list_opt:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        ticker_stats[ticker] = {
            'profit': -999,
            'wins': 0,
            'loses' : 0,
        }
        if df.shape[0] > 0:
          # profit, wins, loses= one_stock_simulation(df, ticker, mode='full')
          profit, wins, loses= one_stock_simulation(df, ticker,
                                                     mode='train',
                                                     _gain_coef=1.005,
                                                     _lose_coef=0.95)
          total_profit += profit
          total_loses += loses
          total_wins += wins
          ticker_stats[ticker] = {
            'profit': profit,
            'wins': wins,
            'loses' : loses,
          }
        else:
          stock_name_list_opt.remove(ticker)
      
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}', color = 'red')
    c.print(f'total_profit_list: {total_profit_list}', color='orange')
    if optimize_profit:
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
    else:
      vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    # np.max(total_profit_list), np.argmax(total_profit_list)

  parameters_list_best = {}
  for parameter in parameters_list_search:
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    
    parameters_list_best[parameter] = vars()[parameter]
    
  print(parameters_list_best)
  print(max(total_profit_list), min(total_loses_list))

  best_profit_improved = max(total_profit_list) > best_profit
  best_profit = max(total_profit_list) if best_profit_improved else best_profit

  # Optimization which stocks to use:
  for ticker in stock_name_list_opt:
    if ticker_stats[ticker]['wins'] / (ticker_stats[ticker]['loses'] + 1) < 3 \
        or ticker_stats[ticker]['profit'] < 0:
      stock_name_list_opt.remove(ticker)
  c.print(f'Current optimal stock list len is {len(stock_name_list_opt)}, list is {stock_name_list_opt}', color='green')


parametr is bull_trend_coef value is 1.01:
Total Profit is 19808.17, total loses is 996
parametr is bull_trend_coef value is 1.02:
Total Profit is 19719.85, total loses is 982
parametr is bull_trend_coef value is 1.03:
Total Profit is 19395.03, total loses is 967
parametr is bull_trend_coef value is 1.04:
Total Profit is 19226.04, total loses is 951
parametr is bull_trend_coef value is 1.05:
Total Profit is 19641.77, total loses is 932
parametr is bull_trend_coef value is 1.06:
Total Profit is 19591.61, total loses is 915
parametr is bull_trend_coef value is 1.07:
Total Profit is 20033.11, total loses is 895
parametr is bull_trend_coef value is 1.08:
Total Profit is 19920.78, total loses is 885
parametr is bull_trend_coef value is 1.09:
Total Profit is 19688.74, total loses is 876
parametr is bull_trend_coef value is 1.1:
Total Profit is 19443.74, total loses is 868
parametr is bull_trend_coef value is 1.11:
Total Profit is 19632.07, total loses is 856
parametr is bull_trend_coef value

# SOME RESULTS, HISTORY LOG

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
6056.55094263172 1011
Current optimal stock list len is 127

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
7049.139814422607 599
Current optimal stock list len is 99,

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 10, 'bull_trend_coef': 1.12}
8616.535972952845 423
Current optimal stock list len is 97

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
8455.558336365704 662
Current optimal stock list len is 71

v1:
{'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
5762.099450752256 170 , number_red_candles= 6

In [94]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)

191

In [9]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

# Current_optimal_stock_list 
# stock_name_list_opt = ['JPM', 'UNH', 'AVGO', 'HD', 'PEP', 'COST', 'CRM', 'WMT',
#                                 'MCD', 'ABT', 'TXN', 'WFC', 'INTC', 'RTX', 'HON', 'SPGI',
#                                   'BA', 'AMGN', 'AMAT', 'SBUX', 'PLD', 'MS', 'DE', 'AXP', 
#                                   'ADI', 'TJX', 'MMC', 'CVS', 'SCHW', 'TMUS', 'ETN', 'CI', 
#                                   'BSX', 'AON', 'ITW', 'SNPS', 'KLAC', 'CDNS', 'WM', 'HCA',
#                                     'FDX', 'MMM', 'CMG', 'GM', 'MCK', 'NXPI', 'NSC',
#                                       'HUM', 'DXCM', 'PH', 'MPC', 'ROP', 'MCHP', 'MSI', 'GD',
#                                         'JCI', 'SRE', 'ADSK', 'AZO', 'TDG', 'AJG', 'TT', 'PCAR',
#                                           'COF', 'CTAS', 'AIG', 'ANET', 'SPG', 'MSCI', 'ROK']

stock_name_list_opt = [
'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
]

len(stock_name_list_opt), len(stock_name_list)


parameter is_near_global_max_prt best value is 96
parameter distance_from_last_top best value is 0
parameter last_top_ratio best value is 1
parameter RIV best value is 0.15
parameter buy_ratio_border best value is 0
parameter bull_trend_coef best value is 1.12


(110, 188)

In [18]:
len(stock_name_list), len(stock_name_list_opt)

(191, 110)

In [104]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
# print("TRAIN RESULTS")
# total_profit = 0
# total_loses = 0
# total_wins = 0
# for ticker in stock_name_list_opt:
#   file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
#   path = os.path.join(parent_path, folder_saved_models, file_name)
#   df = load_data_one_stock(path)
#   profit, wins, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=False)
#   total_profit += profit
#   total_loses += loses
#   total_wins += wins
# print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / total_loses + 1:.2f}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
# for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=False)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / (total_loses + 1):.2f}')
  else:
    print(f'{ticker} empty df')


TEST RESULTS
--------------------------------------------------
Simulation results for ticker JPM:
{'number success': 26, 'number fails': 3, 'number wins': 26, 'proportion': 6.5, 'profit': 55.73440264892578, 'transactions': [{'type': 'fail', 'buy_price': 150.83, 'sell_price': 142.65, 'ratio': 10.85, 'buy_time': Timestamp('2023-08-17 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-08 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 144.71, 'sell_price': 145.62, 'ratio': 1.16, 'buy_time': Timestamp('2023-09-11 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-12 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 145.43, 'sell_price': 146.52, 'ratio': 15.11, 'buy_time': Timestamp('2023-09-25 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-25 15:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 145.86, 'sell_price': 147.94, 'ratio': 3.87, 'buy_time': Ti

In [129]:
print("FULL RESULTS OPTIMIZATION")
# parameters:
lose_coef = 0.95
gain_coef = 1.005
# gain_coef = 1.02
use_optimal_stock_list = False
use_stock_name_list_selected = True
mode = 'full'
#
total_profit = 0
total_loses = 0
total_wins = 0

if use_optimal_stock_list:
  current_stock_name_list = stock_name_list_opt
else:
  current_stock_name_list = stock_name_list

if use_stock_name_list_selected:
  current_stock_name_list = stock_name_list_selected

# current_stock_name_list = ['CME', 'AMT']

df_opt = pd.DataFrame(columns=['ticker','profit', 'wins', 'loses', 'proportion'])

for ticker in current_stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)

  # if ticker in opt_stocks_for_bear_trend:
  #   gain_coef = 1.02
  # else:
  #   gain_coef = 1.005

  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker,
                                               mode=mode,
                                               _gain_coef=gain_coef,
                                               _lose_coef=lose_coef, 
                                               print_sim_results=True, 
                                               print_trans_history=False)
    if loses == 0:
      proportion = wins
    else:
      proportion = wins / loses
    df2 = pd.DataFrame({'ticker': [ticker],'profit': [profit], 'wins': [wins], 'loses': [loses], 'proportion': [proportion]})
    df_opt = pd.concat([df_opt, df2], ignore_index=True)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    if total_loses == 0:
      total_proportion = total_wins
    else:
      total_proportion = total_wins / total_loses
  else:
    print(f'{ticker} empty df')
c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_proportion:.2f}', color = 'blue')
c.print('Parameters are:', color = 'blue' )
c.print(f'gain_coef is {gain_coef}; lose_coef is {lose_coef}; use_optimal_stock_list is {use_optimal_stock_list}, use_selected_optimal_stock_list is \
{use_stock_name_list_selected} mode is {mode}', color = 'blue')

FULL RESULTS OPTIMIZATION
--------------------------------------------------
Simulation results for ticker BA:
{'number success': 52, 'number fails': 3, 'number wins': 52, 'proportion': 13.0, 'profit': 672.2523241882327, 'transactions': [{'type': 'success', 'buy_price': 168.72, 'sell_price': 172.18, 'ratio': 7.92, 'buy_time': Timestamp('2022-08-12 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-15 11:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.98, 'buy_gain_coef': 1.02}, {'type': 'fail', 'buy_price': 168.22, 'sell_price': 164.13, 'ratio': 15.77, 'buy_time': Timestamp('2022-08-17 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-19 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.98, 'buy_gain_coef': 1.02}, {'type': 'success', 'buy_price': 160.95, 'sell_price': 168.81, 'ratio': 10.71, 'buy_time': Timestamp('2022-08-23 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-25 09:30:00-0400', tz='America/

C:\Users\IvanSpok\AppData\Local\Temp\ipykernel_13532\1350040760.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_opt = pd.concat([df_opt, df2], ignore_index=True)


--------------------------------------------------
Simulation results for ticker DHI:
{'number success': 43, 'number fails': 8, 'number wins': 43, 'proportion': 4.777777777777778, 'profit': 446.5081299514771, 'transactions': [{'type': 'fail', 'buy_price': 79.3, 'sell_price': 77.26, 'ratio': 5.05, 'buy_time': Timestamp('2022-08-12 14:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-16 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.98, 'buy_gain_coef': 1.02}, {'type': 'fail', 'buy_price': 76.69, 'sell_price': 74.9, 'ratio': 8.66, 'buy_time': Timestamp('2022-08-18 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-19 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef': 0.98, 'buy_gain_coef': 1.02}, {'type': 'success', 'buy_price': 73.29, 'sell_price': 75.12, 'ratio': 8.2, 'buy_time': Timestamp('2022-08-23 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-24 09:30:00-0400', tz='America/New_York'), 'buy_lose_coef

Results:
1. use ordered selected stock list;
2. Use danamic gain coeficient'


In [101]:
df_opt = df_opt.sort_values(by='profit', ascending=False)
print(df_opt)
df_opt.to_csv('df_opt.csv')

    ticker      profit wins loses  proportion
52      BA  673.455899   51     3   17.000000
160    OXY  603.052024   58     5   11.600000
187    DHI  419.083508   40     8    5.000000
166     ON  412.661394   51    10    5.100000
98    PANW  395.511723   41     8    5.125000
..     ...         ...  ...   ...         ...
78       C -202.906399   36    12    3.000000
38      VZ -291.425342   25    13    1.923077
93      SO -301.359807   25    11    2.272727
138   FTNT -355.170629   33    14    2.357143
178      O -395.342735   19    13    1.461538

[189 rows x 5 columns]


In [103]:
len(stock_name_list_selected)

111

In [113]:
print(stock_name_list_selected.to_list())

['BA', 'OXY', 'DHI', 'ON', 'PANW', 'AMD', 'MCHP', 'BSX', 'INTU', 'HLT', 'NXPI', 'AIG', 'EL', 'LLY', 'USB', 'AMAT', 'ADI', 'ANET', 'DXCM', 'LRCX', 'EOG', 'GE', 'MU', 'PSX', 'NUE', 'AVGO', 'WMB', 'TJX', 'SNPS', 'WMT', 'KLAC', 'MAR', 'SBUX', 'ECL', 'CDNS', 'EMR', 'TT', 'IDXX', 'VLO', 'TXN', 'F', 'ABBV', 'MPC', 'CAT', 'CSCO', 'NFLX', 'JCI', 'TDG', 'MRK', 'MRNA', 'DE', 'NOW', 'TMUS', 'GM', 'WELL', 'ETN', 'ICE', 'WM', 'CME', 'PCAR', 'CTAS', 'MSI', 'GILD', 'SLB', 'CMCSA', 'ROP', 'ADM', 'LOW', 'QCOM', 'VRTX', 'MO', 'EXC', 'CI', 'JNJ', 'CL', 'STZ', 'CMG', 'MMM', 'SCHW', 'GOOG', 'PH', 'LMT', 'HON', 'PEP', 'COP', 'CRM', 'MSCI', 'UNP', 'APD', 'ADP', 'CVS', 'GS', 'HUM', 'ADSK', 'TEL', 'IBM', 'ROK', 'MNST', 'CVX', 'ITW', 'ADBE', 'PM', 'SPG', 'TGT', 'PYPL', 'APH', 'FDX', 'CSX', 'SHW', 'TFC', 'UNH']


In [110]:
stock_name_list_selected = df_opt.loc[df_opt['profit'] > 40]['ticker'].sort_values(by='profit', ascending=False)
stock_name_list_selected.to_list()
# stock_name_for_check = [
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
# 'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
# 'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
# 'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
# 'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
# 'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
# 'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
# 'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
# 'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
# ]
# (stock_name_for_check == stock_name_list_selected).all()
opt_stocks_for_bear_trend = ['BA', 'INTU', 'MCHP', 'LLY', 'DHI', 'ANET', 'AIG', 'NUE', 'MAR', 'OXY', 'ON',
  'GE', 'AMAT', 'NXPI', 'SNPS', 'UNP', 'KLAC', 'BSX', 'MSI', 'CRM', 'CAT', 'ADI',
    'ETN', 'JCI', 'HLT', 'CSCO', 'WMT', 'TDG', 'TT', 'ECL', 'LOW', 'ADSK', 'TJX',
    'VRTX', 'APH', 'ABBV', 'STZ', 'SBUX', 'DE', 'MRK', 'CTAS', 'MNST', 'CME', 'MO', 'TXN', 'ITW']

TypeError: Series.sort_values() got an unexpected keyword argument 'by'

In [136]:
def test():
  return 2, 3
a = test()
a
a is not None

True

In [120]:
df_opt_bear_trend = df_opt.sort_values(by='proportion', ascending=False)

df_opt_bear_trend = df_opt_bear_trend.loc[df_opt_bear_trend['proportion'] >= 3].sort_values(by='profit', ascending=False)
print(df_opt_bear_trend)

    ticker      profit wins loses  proportion
0       BA  674.602803   38     6    6.333333
8     INTU  657.527402   40     7    5.714286
6     MCHP  599.032862   41     8    5.125000
13     LLY  587.327168   26     4    6.500000
2      DHI  573.843221   36     8    4.500000
17    ANET  563.029703   36    12    3.000000
11     AIG  491.698669   32     7    4.571429
24     NUE  465.124099   39    12    3.250000
31     MAR  456.696729   33     6    5.500000
1      OXY  455.211293   36    10    3.600000
3       ON  444.715145   45    13    3.461538
21      GE  440.864404   28     5    5.600000
15    AMAT  410.183924   36    11    3.272727
10    NXPI  404.862433   38    10    3.800000
28    SNPS  376.630274   33    10    3.300000
87     UNP  360.880210   23     6    3.833333
30    KLAC  358.535270   32     8    4.000000
7      BSX  349.135627   20     4    5.000000
61     MSI  348.189323   26     5    5.200000
85     CRM  330.056228   33    11    3.000000
43     CAT  329.017333   26     7 

In [123]:
print(df_opt_bear_trend['ticker'].to_list())

['BA', 'INTU', 'MCHP', 'LLY', 'DHI', 'ANET', 'AIG', 'NUE', 'MAR', 'OXY', 'ON', 'GE', 'AMAT', 'NXPI', 'SNPS', 'UNP', 'KLAC', 'BSX', 'MSI', 'CRM', 'CAT', 'ADI', 'ETN', 'JCI', 'HLT', 'CSCO', 'WMT', 'TDG', 'TT', 'ECL', 'LOW', 'ADSK', 'TJX', 'VRTX', 'APH', 'ABBV', 'STZ', 'SBUX', 'DE', 'MRK', 'CTAS', 'MNST', 'CME', 'MO', 'TXN', 'ITW']


In [ ]:
# Version 1.0
# stock_name_list_opt = [
#   'GOOG', 'JPM', 'XOM', 'UNH', 'AVGO', 'LLY', 'COST',
#   'CRM', 'TMO', 'NFLX', 'TXN', 'INTU', 'NKE', 'QCOM',
#   'BA', 'AMGN', 'MDT', 'PLD', 'MS', 'GS', 'LMT', 'ADI', 'TJX', 'ELV', 'C', 'CVS', 'VRTX', 'SCHW', 'LRCX',
#   'TMUS', 'ETN', 'ZTS', 'CI', 'FI', 'EQIX', 'DUK', 'MU',
#   'AON', 'ITW', 'SNPS', 'KLAC', 'CL', 'WM', 'HCA', 'MMM',
#   'CMG', 'EW', 'GM', 'MCK', 'NSC', 'PH', 'MPC', 'ROP', 
#   'MCHP', 'USB', 'CCI', 'MAR', 'MSI', 'GD', 'JCI', 'PSX', 
#   'SRE', 'ADSK', 'AJG', 'TEL', 'TT', 'PCAR', 'OXY', 'CARR',
#   'IDXX', 'GIS', 'CTAS', 'AIG', 'ANET', 'BIIB', 'SPG', 'MSCI', 'DHI'
# ]

# Version 2.0
# stock_name_list_opt = [
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
# 'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
# 'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
# 'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
# 'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
# 'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
# 'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
# 'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
# 'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
# ]

# settings for buy condition Version 1.0
# is_near_global_max_prt = 80
# distance_from_last_top  = 0
# last_top_ratio = 1
# RIV  = 0.25
# buy_ratio_border = 9
# bull_trend_coef = 1.12
# number_tries_to_submit_order = {}
#
# settings for buy condition Version 2.0
# is_near_global_max_prt = 96
# distance_from_last_top  = 0
# last_top_ratio = 1
# RIV  = 0.15
# buy_ratio_border = 0
# bull_trend_coef = 1.12
# number_tries_to_submit_order = {}
#